In [1]:
#| default_exp rf_reg

# Random forest regression

In [2]:
#|hide
from nbdev.showdoc import *

In [3]:
#|hide
#|export
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV,gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args
from sklearn.model_selection import cross_val_score

from sklearn.metrics import r2_score

from ML_projects import const_vals as CONST


In [10]:
#|export

class TrainRFReg():
  def __init__(self,
      train_test_data : list , # list with train/test data , in this order : [x_train,x_test,y_train,y_test]
      hyper_method : str , #hyperparameter tunning method. accepts : 'randomized' 'bayesian' , 'bayesian continous'
      hyper_params : dict = CONST.RANDOM_GRID_RFR , #parameters for hyperparameter tunning
      space : list = CONST.SPACE_RFR , #space for continues bayesian continous search 
      ):
     self.hyper_method = hyper_method
     self.hyper_params = hyper_params

     self.x_train = train_test_data[0]
     self.x_test = train_test_data[1]
     self.y_train = train_test_data[2]
     self.y_test = train_test_data[3]
     
     # space for bayesian continous searching
     self.space = space

     #generate random forest regressor 
     self.model = RandomForestRegressor()


     self.best_params = self.hyperparameter()

  def objective(self):
    '''
    Scitkit Learn Optimize requires an objective function to minimize.
    We use the average of cross-validation mean absolute errors as 
    the objective function (also called cost function in optimization)
    '''
    self.model.set_params(**self.hyper_params)

    return -np.mean(cross_val_score(self.model, self.x_train, 
                                    self.y_train, 
                                    cv=CONST.CV_RFR, 
                                    n_jobs=CONST.N_JOBS,
                                    scoring="neg_mean_absolute_error"))



  def hyperparameter(self):
     
     if self.hyper_method == 'randomized':
       
       rf_random = RandomizedSearchCV(estimator = self.model, 
                                      param_distributions = self.hyper_params,
                                      n_iter = CONST.N_ITERATIONS_RFR,
                                      cv = CONST.CV_RFR, 
                                      verbose=CONST.VERBOSE , 
                                      random_state=CONST.RANDOM_STATE , 
                                      n_jobs = CONST.N_JOBS)
       
       #fit model 
       rf_random.fit(self.x_train, self.y_train)

       #best params
       self.best_params = rf_random.best_params_


     elif self.hyper_method == 'bayesian':
      
      rf_bayes = BayesSearchCV(self.model,
                      search_spaces=self.hyper_params, 
                      n_iter=CONST.N_ITERATIONS_RFR, 
                      cv=CONST.CV_RFR)
      np.int = int
      
       #fit model 
      rf_bayes.fit(self.x_train, self.y_train)

      self.best_params = rf_bayes.best_params_

     elif self.hyper_method == 'bayesian continous':


      self.best_params = gp_minimize(self.objective, 
                                     dimension = list(CONST.SPACE_RFR.keys()),
                                     space = CONST.SPACE_RFR, 
                                     n_calls=CONST.N_CALLS, 
                                     random_state=CONST.RANDOM_STATE)

     return self.best_params
       
       


In [9]:
list(CONST.SPACE_RFR.keys())

['bootstrap',
 'max_depth',
 'max_features',
 'min_samples_leaf',
 'min_samples_split',
 'n_estimators']

In [7]:

      #  def train_model(self):
      #         # create base model
      #         rf = RandomForestRegressor()

      #         rf_random = RandomizedSearchCV(estimator = rf, 
      #                                        param_distributions = CONST.random_grid_rf,
      #                                          n_iter = CONST.n_iteration_rf,
      #                                            cv = CONST.cv_rf, 
      #                                            verbose=CONST.VERBOSE , 
      #                                            random_state=CONST.RANDOM_STATE , 
      #                                            n_jobs = CONST.N_JOBS)
  
      #           # Fit the random search model
      #         rf_random.fit(self.x_train, self.y_train)
              
      #         #train model 
      #         self.best_rf_params = rf_random.best_params_

      #         #fit best model
      #         self.rf_model = RandomForestRegressor(**self.best_rf_params)

      #         self.rf_model.fit(self.x_train,self.y_train)
      #         #predict

      #         y_pred = self.rf_model.predict(self.x_test)

      #         self.r2 = r2_score(y_pred, self.y_test)



      #         return self.rf_model